In [1]:
import pandas as pd
import os

In [9]:
# Directory containing subdirectories for each housing
# Each subdirectory contains a csv file 
data_path = "../../data/housing_consomation/"
list_housings = os.listdir(data_path)

In [33]:
def extract_housing_info_from_name(housing):
    """
    Extracts the housing information from the csv file
    The info is as follows:
        - The first upper case letter is the type of housing (A : appartement, M : maison)
        - The numbers after the letter represent the surface of the housing
        - The number after the dash represents the number of people living in the housing
    """
    info = housing.split(" ")[1]
    print(f"Housing {info}")
    h_type = info[0]
    info = info.replace(h_type, "")
    surface, nb_people = info.split("-")
    print(f"Housing type : {h_type}")
    print(f"Surface (m²) : {surface}")
    print(f"Number of people : {nb_people}")


In [34]:
extract_housing_info_from_name(list_housings[0])

Housing A110-5
Housing type : A
Surface (m²) : 110
Number of people : 5
